In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
from collections import OrderedDict as od
from threading import Thread
from queuelib import queue
from selenium import webdriver
from datetime import datetime
import time
import random
import json
import csv
import traceback as tb
import re

In [2]:
HOST = 'https://www.superbuy.com.tw/'

In [3]:
index_url = HOST

In [158]:
SuperBuy_crawler()

主食雜糧
雜糧
https://www.superbuy.com.tw/product.php?p_id=1509080001
脫殼台灣紅藜
藜麥
成份：台灣藜
原產地：台灣
保存期限：未開封保存一年
https://www.superbuy.com.tw/product.php?p_id=1609220001
帶殼台灣紅藜
藜麥
成份：台灣紅藜
原產地：台灣
保存期限：未開封保存一年
https://www.superbuy.com.tw/product.php?p_id=1603180010
自然農法三色藜麥(買大送大)
藜麥
成份：三色黎麥
原產地：祕魯
保存期限：1年
https://www.superbuy.com.tw/product.php?p_id=1605170001
帶殼台灣紅藜飯
藜麥
成份：帶殼台灣紅藜、花東優質無毒白米
原產地：台灣
保存期限：未開封6個月
https://www.superbuy.com.tw/product.php?p_id=1701060008
有機轉型脫殼台灣紅藜
藜麥
成份：台灣紅藜
原產地：屏東
保存期限：1年(未開封)
https://www.superbuy.com.tw/product.php?p_id=1603180009
自然農法三色藜麥(買大送小)
藜麥
成份：三色黎麥
原產地：祕魯
保存期限：1年
https://www.superbuy.com.tw/product.php?p_id=1701060009
有機轉型帶殼台灣紅藜
藜麥
成份：台灣紅藜
原產地：屏東
保存期限：1年(未開封)
https://www.superbuy.com.tw/product.php?p_id=1311130005
香Q糯米薏仁
小麥、薏仁
成份：糯米薏仁
原產地：寮國
保存期限：請盡快食用完畢
https://www.superbuy.com.tw/product.php?p_id=1604180017
筍茸
小麥、薏仁
成份：有機麻竹筍、粗鹽
原產地：嘉義 阿里山
保存期限：12個月
https://www.superbuy.com.tw/product.php?p_id=1407180001
德國自然農法北非小米
小麥、薏仁
成份：北非小米
原產地：法國
保存期限：12個月
https://www.super

IndexError: list index out of range

# 上層爬蟲

In [8]:
def SuperBuy_crawler():
    res_index = rq.get(index_url)
    soup_index = bs(res_index.text, 'lxml')
    soup_li = soup_index.li
    
    rows = soup_index.select('.row')
    li_lists = rows[0].find('div')
    a_list = li_lists.select('a')
    
    for a in a_list:
        print(a.text)
        petten = 'category.php'
        pre_small_branch_url = a['href']
        regular_expression = re.match(petten,pre_small_branch_url)
        
        if regular_expression is not None:
            #print(pre_small_branch_url)
            small_branch_url = "{}{}".format(index_url, pre_small_branch_url)
            #print(small_branch_url)
            #pass
            #傳給中層
            small_branch_crawler(small_branch_url)    

# 中層爬蟲

In [112]:
def small_branch_crawler(small_branch_url):
    res = rq.get(small_branch_url)
    soup = bs(res.text, 'lxml')
    
    a_list = soup.select('.product.col-xs-6.col-sm-6.col-md-4 > ul > li > a["href"]')
    
    last_product_url = ''
    
    for a_href in a_list:
        pre_product_url = re.match(r'product\.php\?p\_id',a_href['href'])
        if pre_product_url is not None:
            product_url = "{}{}".format(index_url, a_href['href'])
            
            if product_url != last_product_url:
                print(product_url)
                #pass
                #傳給下層
                #product_url_crawler(product_url)
                test_product_url_crawler(product_url)
                last_product_url = product_url
                #print(last_product_url)

# 下層爬蟲

In [44]:
test_product_url = 'https://www.superbuy.com.tw/product.php?p_id=1212120034'

In [157]:
def test_product_url_crawler(product_url):
    res = rq.get(product_url)
    soup = bs(res.text, 'lxml')
    title = soup.h1.text
    #標題
    print(title)
    #類別
    category = soup.select('.productinfo > p > a')[0].text
    print(category)

    top_box = soup.select('.col-sm-12')[1]
    #成分
    re_component_pattern = r"成份：[\u4e00-\u9fa5\u3001-\u3003\uff1f\uff0c]+.*"
    pre_component = top_box.find('li').text.strip()
    if re.match(re_component_pattern, pre_component):
        print(pre_component)

    #原產地
    re_origin_pattern = r"原產地：[\u4e00-\u9fa5\u3001-\u3003\uff1f\uff0c]+.*"
    pre_origin = top_box.findAll('li')[1].text.strip()
    if re.match(re_origin_pattern, pre_origin):
        print(pre_origin)

    # 保存期限
    re_retention_period_pattern = r"保存期限：\d*[\u4e00-\u9fa5\u3001-\u3003\uff1f\uff0c]+.*"
    pre_retention_period = top_box.findAll('li')[2].text.strip()
    if re.match(re_retention_period_pattern, pre_retention_period):
        print(pre_retention_period)
    print('===========================================')